In [84]:

# http://blog.gregzaal.com/how-to-install-ffmpeg-on-windows/
# pip install pydub

from pydub import AudioSegment,silence
import re
import eyed3 # mp3 tag

# input
from io import BytesIO

# output
import datetime
mp3_outdir = datetime.datetime.now().strftime("%Y.%m")

In [85]:
# build cover image
from PIL import Image, ImageFont, ImageDraw 

text = mp3_outdir
image = Image.open('res/인기가요.jpg')
width, height = image.size 

draw = ImageDraw.Draw(image, 'RGBA')
font = ImageFont.truetype('arial', 32)

draw.rectangle((0, height-55, width, height-10),  fill=(0,0,0,100))
draw.text((width-130, height-50), text, fill='white', font=font, stroke_width=2, stroke_fill='black')

ALBUM_COVER = text+'.cover.jpg'
image.save(ALBUM_COVER, quality=100, subsampling=0)

In [86]:
# trackinfos
trackinfos = '''
1. 00:00:01 그때 그 순간 그대로 - WSG워너비(가야G)

2. 00:03:42 보고싶었어 - WSG워너비(4FIRE)

3. 00:07:11 LOVE DIVE - IVE(아이브)POP!

4. 00:09:59 POP! - 나연

5. 00:12:56 That That - 싸이(PSY)

6. 00:16:04 TOMBOY - (여자)아이들

7. 00:18:45 정이라고 하자 - 서동현

8. 00:21:55 사랑인가봐 - 멜로망스

9. 00:25:00 나의 X에게 - 경서

10. 00:28:09 Girls - aespa

11. 00:32:08 그라데이션 - 10cm

12. 00:35:30 봄여름가을겨울 - 빅뱅

13. 00:38:38 Love Me -비오

14. 00:41:30 Clink Clink (클링클링)  - WSG워너비

15. 00:44:55 FEARLESS - 르세라핌

16. 00:47:43내가 아니라도 - 주호

17. 00:51:50 Love Story - 볼빨간사춘기

18. 00:55:07 SNEAKERS - ITZY 

19. 00:58:06 취중고백 - 멜로망스

20. 01:01:55 Feel My Rhythm - 레드벨벳

21. 01:05:24 신호등 - 이무진

22. 01:09:16 ELEVEN - IVE 

23. 01:12:17 도깨비불 - aespa

24. 01:15:31 다정히 내 이름을 부르면 - 경서예지

25. 01:19:24 열이 올라요 - 선미 

26. 01:22:36 아무래도 난 - 주시크

27, 01:26:10 사랑은 늘 도망가 - 임영웅

28. 01:30:13 GANADARA - 박재범

29. 01:33:38 STAY - 블랙핑크

30. 01:35:59 Left and RIght - BTS

31. 01:38:33 Yet to Come - BTS

32. 01:41:47 바보에게 바보가 - 이석훈

33. 01:45:36 MY BAG - 여자아이들

34. 01:48:18 INVU - 태연

35. 01:51:42 듣고 싶을까 - MSG워너비

36. 01:54:29 늦은 밤 헤어지긴 너무 아쉬워 - 케이시

37. 01:57:15 Next Level - aespa

38. 02:00:55 That's Hiarious - Charlie Puth

39. 02:03:25 너의 모든 순간 - 성시경

40. 02:07:27 I LOVE YOU - WINNER

41. 02:10:23 HOT - 세븐틴

42. 02:13:41 팡파레 - 다비치

43. 02:17:10 해요(2022) - #안녕

44. 02:21:05 너를 생각해 - 주시크

45. 02:24:42 SMILEY - 최예나

46. 02:27:34 고백하는 취한밤에 - 임재현

47. 02:31:41 언제나 사랑해 - 케이시

48. 02:35:05 내 손을 잡아 - 아이유

49. 02:38:20 Permissin to Dance - BTS

50. 02:41:28 WORLD - 세븐틴
'''

regexfmt, i_sec, i_title, i_artist = r'\d+. ?([^\s]+) ?([^-]+)- ?(.+)', 1, 2, 3  # [00:00] 01 주호 - 내가 아니라도 | Juho - Without me


lines = trackinfos.splitlines()
lines = [l for l in lines if len(l) > 10]
# print(lines)
import re

tracks = []
for l in lines:
    m = re.search(regexfmt, l)
    if m is None:
        print('Invalid:', l)
    else:
        startsec = m[i_sec].strip()
        title = m[i_title].strip()
        artist = m[i_artist].strip()
        tracks.append((startsec, title, artist))

print('track# =', len(tracks))
print('startsec, title, artist')
print('\n'.join([str(i) for i in tracks[:3]]))
print('\n'.join([str(i) for i in tracks[-3:]]))

track# = 50
startsec, title, artist
('00:00:01', '그때 그 순간 그대로', 'WSG워너비(가야G)')
('00:03:42', '보고싶었어', 'WSG워너비(4FIRE)')
('00:07:11', 'LOVE DIVE', 'IVE(아이브)POP!')
('02:35:05', '내 손을 잡아', '아이유')
('02:38:20', 'Permissin to Dance', 'BTS')
('02:41:28', 'WORLD', '세븐틴')


In [ ]:
# loading source mp3.
from pydub import AudioSegment,silence

youtube_mp3 = r'20220729.mp3'
print('loading', youtube_mp3)
if youtube_mp3.endswith('.webm'):
    with open(youtube_mp3, 'rb') as fh:
        opus_data = BytesIO(fh.read())
        srcmp3 = AudioSegment.from_file(opus_data, codec="opus")
else:
    srcmp3 = AudioSegment.from_mp3(youtube_mp3)

print('total seconds:', int(len(srcmp3) / 1000))



In [ ]:
# export mp3 with near endtick detection.

#
# seconds
#
def get_sec(time_str):
    if time_str is None:
        return 0
        
    time_str = re.sub('[\[\]]', '', time_str)
    tokens = time_str.split(':')
    if len(tokens) == 3:
        h, m, s = tokens
        return int(h) * 3600 + int(m) * 60 + int(s)

    m, s = tokens
    sec = int(m) * 60 + int(s)
    return 0 if sec < 10 else sec


def tick_to_str(tick):
    sec = int(tick/1000)
    min = sec//60
    sec -= min*60
    return f"{int(min):02d}:{int(sec):02d}"

#
# detect end tick
#
def get_endtick(mp3, endsec, MARGIN_SEC=5):
    search_start = (endsec-MARGIN_SEC)*1000
    search_end = (endsec+MARGIN_SEC)*1000
    mute_ranges = silence.detect_silence(mp3[search_start:search_end], min_silence_len=1000, silence_thresh=mp3.dBFS-32)

    if len(mute_ranges) == 0:
        print(f'No silence: {search_start}~{search_end}, {tick_to_str(search_end-starttick)}')
        return endsec*1000

    max_range = mute_ranges[0]
    for range in mute_ranges[1:]:
        if range[1] - range[0] > max_range[1] - max_range[0]:
            max_range = range

    margin_tick = int((max_range[1] - max_range[0]) * 2 / 3)
    endtick = search_start + max_range[0] + margin_tick

    print('range:', max_range, 'tick:', starttick, endtick)
    assert endtick > starttick, f'invalid endtick: {endtick}, starttick={starttick}'
    return endtick


def get_endtick_range(mp3, starttick, MIN_SONG_SEC=2*60, MAX_SONG_SEC=4*60):
    search_start = starttick+MIN_SONG_SEC*1000
    search_end = starttick + MAX_SONG_SEC*1000
    mute_ranges = silence.detect_silence(mp3[search_start:search_end], min_silence_len=1500, silence_thresh=mp3.dBFS-32)

    print('starttick:', starttick, 'search range:', search_start, search_end)

    if len(mute_ranges) == 0:
        print(f'No silence: {starttick}~{search_end}, {(search_end-starttick)//1000}sec')
        return get_endtick_range(mp3, search_end-10*1000, MIN_SONG_SEC=0, MAX_SONG_SEC=60) # continous search.

    print(f'mute_ranges({MIN_SONG_SEC}): {search_start}~{search_end}, {tick_to_str(search_end-search_start + MIN_SONG_SEC*1000)}', mute_ranges)
    max_range = mute_ranges[0]
    for range in mute_ranges[1:]:
        if (range[1] - range[0]) > (max_range[1] - max_range[0]): # max? first?
            max_range = range

    margin_tick = int((max_range[1] - max_range[0]) * 2 / 3)
    endtick = search_start + max_range[0] + margin_tick
    if (endtick+10*1000) > search_end:  # Too close to end, research
        print(f'Too close to end, research: {starttick}~{search_end}, {tick_to_str(search_end-starttick)}')
        return get_endtick_range(mp3, search_end-15*1000, MIN_SONG_SEC=0, MAX_SONG_SEC=60) # continous search.

    print('range:', max_range, 'tick:', starttick, endtick)
    assert endtick > starttick, f'invalid endtick: {endtick}, starttick={starttick}'
    return endtick

#
# export
#
def add_cover_to_mp3(mp3file, title, artist, cover = None):
    if ALBUM_COVER:
        cover = ALBUM_COVER

    audio = eyed3.load(mp3file)
    audio.initTag()
    audio.tag.title = title
    audio.tag.artist = artist
    audio.tag.album_artist = artist
    audio.tag.album = artist
    if cover is not None:
        audio.tag.images.set(3, open(cover, 'rb').read(), 'image/jpeg')
    audio.tag.save(version=eyed3.id3.ID3_V2_3)

def export_mp3(ith, srcmp3, starttick, endtick, title, artist, cover=None):
    filename = re.sub('[\/:*?"<>|]', ',', title + "-" + artist + ".mp3") # valid filename
    mp3file = f'{mp3_outdir}\\{ith}-{filename}'
    track_sound = srcmp3[starttick: endtick] if endtick > 0 else srcmp3[starttick:]
    track_sound.export(mp3file, format='mp3').close()
    
    add_cover_to_mp3(mp3file, title, artist, cover)
    return mp3file

#
# debug
#
if 0: 
# 1. 00:00:01 그때 그 순간 그대로 - WSG워너비(가야G)
# 2. 00:03:42 보고싶었어 - WSG워너비(4FIRE)
# 3. 00:07:11  POP! - 나연
    starttick = 0
    endtick = get_endtick(srcmp3, get_sec('00:03:42'))
    print('endtick=', endtick, f'{tick_to_str((endtick-starttick))}')
    export_mp3(0, srcmp3, starttick, endtick, 'title', 'artist')
    
    starttick = endtick+1
    endtick = get_endtick(srcmp3, get_sec('00:07:11'))
    print('endtick=', endtick, f'{tick_to_str((endtick-starttick))}')
    export_mp3(2, srcmp3, starttick, endtick, 'title', 'artist')
    raise


#
# body
#
starttick = 0
# tracks = [(startsec, title, artist)+]
for i, (startsec, title, artist) in enumerate(tracks,1):
    try:
        endsec = get_sec(tracks[i][0])
    except:
        endsec = 0

    print()
    print('try', title, startsec, endsec)

    endtick = get_endtick_range(srcmp3, starttick) if endsec == 0 else get_endtick(srcmp3, endsec)
    print( export_mp3(i, srcmp3, starttick, endtick, title, artist) )
    starttick = endtick+1

    if i==50:break
